In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as functional

In [64]:
class Likelihood_Loss(nn.Module):
    """
    Takes:
    - activation_batch of size (batch_size, 1, x, y) describing the predicted probability distribution for fixations 
      (how likely is each image to be fixated) over all pixels of an (x,y)-image 
    - target_batch of size (batch_size, 1, x, y) describing the amount of fixations every pixel got (0 for no fixation, 1 for
      one fixation, 2 for two fixations and so on)
      
    Gives:
    - negative log-likelihood_loss: likelihood of fixation locations in the target assuming the predicted probability
      distribution. If on pixel has more than one fixation, its likelihood is counted as many times. For the implementation
      this means that we can just multipy the predicted probability distribution with the target map. In doing so, we evaluate
      the predicted probability density in the fixated locations, counting a location more than once if it was fixated more 
      than once
    - mean of negative log-likelihood_loss over the given batch is returned
    """
    
    def __init__(self):
        super(Likelihood_Loss, self).__init__()
        
    def forward(self, activation_batch, target_batch):
        x = torch.log(activation_batch * target_batch)
        x = torch.sum(torch.sum(x, dim=-1), dim=-1)
        return -1*torch.mean(x) #mean over the batch

    
criterion = Likelihood_Loss()

In [15]:
class Likelihood_Loss(nn.Module):
    """
    Takes:
    - activation_batch of size (batch_size, 1, x, y) describing the predicted probability distribution for fixations 
      (how likely is each image to be fixated) over all pixels of an (x,y)-image 
    - target_batch of size (batch_size, 1, x, y) describing the amount of fixations every pixel got (0 for no fixation, 1 for
      one fixation, 2 for two fixations and so on)
      
    Gives:
    - negative log-likelihood_loss: log-likelihood of fixation locations in the target assuming the predicted probability
      distribution. If on pixel has more than one fixation, its likelihood is counted as many times. For the implementation
      this means that we can just multipy the predicted probability distribution with the target map. In doing so, we evaluate
      the predicted probability density in the fixated locations, counting a location more than once if it was fixated more 
      than once
    - mean of negative log-likelihood_loss over the given batch is returned
    """
    
    def __init__(self):
        super(Likelihood_Loss, self).__init__()
        
    def forward(self, activation_batch, target_batch):
        #print(activation_batch[0])
        #print(torch.sum(activation_batch[0]))
        #Wieso log-Likelihood? --> Kümmerer et al.
        #Wie ohne Offset stabil machen?
        #x = activation_batch * target_batch
        #x = x[x.nonzero().detach_()]
        #x = -torch.log(activation_batch * target_batch + 1e-05)
        #x = -(activation_batch * target_batch)
        #print(torch.sum(x))
        #x = torch.sum(torch.sum(x, dim=-1), dim=-1)
        #print(torch.mean(x))
        #return torch.mean(x) #mean over the batch
        
        #x = torch.log(activation_batch + 1e-10) + torch.log(target_batch + 1e-10)
        #x = torch.sum(torch.sum(x, dim=-1), dim=-1)
        #print(torch.mean(x))
        #return torch.mean(x)
        
        loss = 0
        for h in range(activation_batch.size(0)):
            for (i,j) in target_batch[h]:
                if (i,j) == (-1000,-1000):
                    break
                loss += torch.log(activation_batch[h,0,i,j] + 1e-10)
        loss /= activation_batch.size(0)
        #print(loss)
        return loss * -1

In [19]:
criterion = Likelihood_Loss()
criterion(torch.randn(8,1,5,5), torch.randn(8,5,2).long())

tensor(nan)

In [34]:
a = torch.randn(8,3,4,4)
m = nn.Softmax2d()
m(a)

tensor([[[[0.4548, 0.1118, 0.0370, 0.8761],
          [0.4780, 0.1519, 0.1048, 0.1138],
          [0.5016, 0.3614, 0.6444, 0.2943],
          [0.3240, 0.2400, 0.1644, 0.6921]],

         [[0.2292, 0.3721, 0.8452, 0.0470],
          [0.4249, 0.2761, 0.5593, 0.4710],
          [0.3816, 0.2314, 0.2987, 0.5263],
          [0.4711, 0.5550, 0.6006, 0.2744]],

         [[0.3160, 0.5161, 0.1178, 0.0769],
          [0.0971, 0.5720, 0.3359, 0.4152],
          [0.1168, 0.4072, 0.0569, 0.1793],
          [0.2049, 0.2050, 0.2349, 0.0335]]],


        [[[0.4739, 0.5480, 0.1776, 0.1216],
          [0.4542, 0.2430, 0.1301, 0.3771],
          [0.4173, 0.2845, 0.4166, 0.3313],
          [0.0319, 0.3617, 0.1070, 0.3660]],

         [[0.0664, 0.2292, 0.4134, 0.6667],
          [0.3911, 0.2133, 0.8382, 0.2395],
          [0.2644, 0.5553, 0.5252, 0.2442],
          [0.1197, 0.5685, 0.2306, 0.5204]],

         [[0.4597, 0.2228, 0.4090, 0.2117],
          [0.1546, 0.5437, 0.0318, 0.3834],
          [0.3183, 0

In [53]:
x = torch.randn(8,1,100,100)
y = nn.Softmax(2)(x.view(*x.size()[:2], -1)).view_as(x)
#*x.size()[:2] gibt (8,1), also (8,1,-1) als Dimensionen also (8,1,16). Dann Softmax über dim=2
y[2,0].sum()
y.size()

torch.Size([8, 1, 100, 100])

In [23]:
a = torch.randn(4,4)
b = functional.softmax(a, dim=-1)
print(b)
torch.sum(functional.softmax(b, dim=-1))

tensor([[0.1250, 0.0750, 0.5457, 0.2542],
        [0.0623, 0.1755, 0.5490, 0.2131],
        [0.0455, 0.5870, 0.2314, 0.1361],
        [0.0398, 0.1136, 0.4456, 0.4009]])


tensor(4.0000)

In [31]:
target = torch.randn(8,100,100)
summe = torch.sum(torch.sum(target, -1), -1)
print(target.size())
target = target[0]/summe[0]
target.size()

torch.Size([8, 100, 100])


torch.Size([100, 100])

In [24]:
torch.sum(torch.sum(target,-1), -1).size()

torch.Size([8])

In [32]:
loss = nn.CrossEntropyLoss()
inputs = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(inputs, target)

In [35]:
torch.empty(3, dtype=torch.long)#.random_(5)

tensor([139864837979016,        67909120,              32])

In [39]:
batch_size = 1
c, h, w = 1, 10, 10
nb_classes = 3
x = torch.randn(batch_size, c, h, w)
target = torch.empty(batch_size, h, w, dtype=torch.long).random_(nb_classes)
print(target)

model = nn.Conv2d(c, nb_classes, 3, 1, 1)
criterion = nn.CrossEntropyLoss()

output = model(x)
loss = criterion(output, target)
loss.backward()

tensor([[[2, 0, 0, 1, 0, 1, 2, 0, 2, 2],
         [0, 2, 0, 2, 0, 2, 0, 2, 1, 0],
         [0, 0, 2, 2, 0, 1, 0, 0, 0, 1],
         [2, 1, 0, 1, 1, 1, 1, 1, 2, 2],
         [1, 2, 1, 2, 1, 2, 1, 1, 1, 1],
         [1, 0, 0, 0, 1, 0, 2, 0, 0, 0],
         [1, 1, 1, 2, 2, 1, 2, 0, 2, 2],
         [1, 0, 1, 2, 1, 2, 0, 0, 0, 0],
         [2, 2, 0, 0, 2, 1, 2, 2, 0, 1],
         [2, 0, 0, 2, 2, 1, 1, 1, 2, 1]]])
